    Index Investing vs Active Investing
    Copyright (C) 2022 Wagner Bertholdo Burghausen
	<https://github.com/wagner-b>, <wagb2b2@gmail.com>

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.

    Disclaimer: This file contains only educational content. There is NO investment advice 
    or financial advice of any kind here. I am NOT a financial advisor, and I take 
    NO responsibility over the consequences of any investing decision you take.

# Index Investing v.s. Active Investing

## 1. Introduction

First, a quick context about index investing and active investing (market timing) for those unfamiliar:

> From [Investopedia](https://www.investopedia.com/terms/p/passiveinvesting.asp): "Index investing in one common passive investing strategy whereby investors purchase a representative benchmark, such as the S&P 500 index, and hold it over a long time horizon. [...]. Passive investment is cheaper, less complex, and often produces superior after-tax results over medium to long time horizons than actively managed portfolios."

> Also from [Investopedia](https://www.investopedia.com/articles/stocks/08/passive-active-investing.asp): "Market timing includes actively buying and selling to try and get into the market at the most advantageous times while avoiding the disastrous times. Research shows that long-term buy-and-hold tends to outperform, where market timing remains very difficult. Much of the market’s greatest returns or declines are concentrated in a short time frame."

In this project, I want to test the above affirmation that "long-term buy-and-hold tends to outperform" active investing strategies. I will analyze real world data over a long time period (2007 - 2022), and I will create a computer algorithm to manage the active investing strategy (market timing), thus removing any human emotional factor, which in many occasions could play a big role in the "generally inferior results" of active investing strategies, compared to index investing.

Please, keep in mind I am not considering any fees or taxes that may be applicable in real life, since there is too much variation depending on where you live and what services you use. I should also remind you that nothing in this file is financial advise of any kind, everything here is just educational content.

## 2. Defining project goals and how to achieve them

#### The goal of this project is to create a computer algorithm to manage an active investing portfolio (based on stock picking and market timing), and compare its returns against those of the S&P 500 index, from 2007 to 2022.

In this project, I will create a computer algorithm that manages a theoretical investing portfolio, and partakes in active trading, chosing between stocks listed on the S&P 500 index (taking into account the changes in the composition of this index over time). I will use arbitrary criteria for which factors will be a trigger for adding, maintaining or removing stocks from the portfolio. Everything will depend on which factors give the best possible returns for the training dataset.

Then, I will run (only once) the final strategy model against the validation dataset (a separate dataset in the relative future compared to the training dataset), and compare its returns with the returns of the S&P 500 index. Although I do not plan on using machine learning in this project, I choose to separate the training and validation datasets in order to be as fair as possible, and to avoid misleading interpretation of the results in case __overfitting__ happens.

> From [IBM](https://www.ibm.com/cloud/learn/overfitting): "**Overfitting** is a concept in data science, which occurs when a statistical model fits exactly against its training data. When this happens, the algorithm unfortunately cannot perform accurately against unseen data, defeating its purpose."

Another important detail is that I will **not** use the current S&P 500 composition to pick stocks in the past, because that would introduce a very serious kind of selection bias in my study, called survivorship bias. To avoid that, I will collect and clean the monthly S&P 500 composition data during the entire time period of this study.

> From [Wikipedia](https://en.wikipedia.org/wiki/Survivorship_bias): "__Survivorship bias__, __survival bias__ or __immortal time bias__ is the logical error of concentrating on the people or things that made it past some selection process and overlooking those that did not, typically because of their lack of visibility. [...]. Survivorship bias is a form of selection bias that can lead to overly optimistic beliefs because multiple failures are overlooked, such as when companies that no longer exist are excluded from analyses of financial performance."

## 3. Collecting the complete dataset of the S&P 500 composition from 2007 to 2022

In [1]:
from calendar import monthrange
from datetime import date
from datetime import datetime
import numpy as np
from pandas_datareader import data as pdr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

I will use the `monthrange` function of the `calendar` library to find what is the last day of the month in each year and month. The examples below shows this functions returns a tuple where the first element is the day of the week (Monday = 0 -> Sunday = 6) of day 1, and the second element is the last day of that month in that year.

In [2]:
monthrange(2006, 10)

(6, 31)

In [3]:
print(monthrange(2006, 10)[1])
print(monthrange(2006, 10)[-1])

31
31


In [4]:
date(2006, 10, 1).weekday()

6

In [5]:
date(2022, 10, 24).weekday()

0

I need to generate a list with the dates of each final business day of the month (so saturdays and sundays should not be on this list) for the entire time period. Below, I created the `return_last_business_day` function to help me with that. It also deals with some special conditions imposed by the source of the S&P 500 composition data used here (ishares website).

In [6]:
def return_last_business_day(YYYY, MM):
    """
        Checks if the last day of the month is Saturday or
    Sunday, and if it is, returns day-1 or day-2, respectively.
    If last day of month is business day, simply return that day.
    I've included special conditions for some years in March.
    """
    
    # Special conditions because of ishares data
    if YYYY == 2013 and MM == 3:
        return 28
    if YYYY == 2018 and MM == 3:
        return 29
    
    # This is the otherwise "normal execution" of the function
    last_day = monthrange(YYYY, MM)[-1]
    
    if date(YYYY, MM, last_day).weekday() == 5:
        return (last_day - 1)
    elif date(YYYY, MM, last_day).weekday() == 6:
        return (last_day - 2)
    else:
        return last_day


# Creating the list with the last business day of each month
end_of_month_dates = [date(2006, 10, 31), date(2006, 11, 30),
                      date(2006, 12, 29)]

for year in range(2007, 2023):
    for month in range(1, 13):
        business_day = return_last_business_day(year, month)
        end_of_month_dates.append(date(year, month,
                                       business_day))

end_of_month_dates

[datetime.date(2006, 10, 31),
 datetime.date(2006, 11, 30),
 datetime.date(2006, 12, 29),
 datetime.date(2007, 1, 31),
 datetime.date(2007, 2, 28),
 datetime.date(2007, 3, 30),
 datetime.date(2007, 4, 30),
 datetime.date(2007, 5, 31),
 datetime.date(2007, 6, 29),
 datetime.date(2007, 7, 31),
 datetime.date(2007, 8, 31),
 datetime.date(2007, 9, 28),
 datetime.date(2007, 10, 31),
 datetime.date(2007, 11, 30),
 datetime.date(2007, 12, 31),
 datetime.date(2008, 1, 31),
 datetime.date(2008, 2, 29),
 datetime.date(2008, 3, 31),
 datetime.date(2008, 4, 30),
 datetime.date(2008, 5, 30),
 datetime.date(2008, 6, 30),
 datetime.date(2008, 7, 31),
 datetime.date(2008, 8, 29),
 datetime.date(2008, 9, 30),
 datetime.date(2008, 10, 31),
 datetime.date(2008, 11, 28),
 datetime.date(2008, 12, 31),
 datetime.date(2009, 1, 30),
 datetime.date(2009, 2, 27),
 datetime.date(2009, 3, 31),
 datetime.date(2009, 4, 30),
 datetime.date(2009, 5, 29),
 datetime.date(2009, 6, 30),
 datetime.date(2009, 7, 31),
 date

As stated previously, I will use the [ishares website](https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf) as the source of the data for the S&P 500 components. This is a very trustworthy source of data, since iShares is in charge of one of the three largest S&P 500 ETFs, the iShares Core S&P 500 ETF (ticker: IVV) that many investors know and buy in order to replicate the returns of the S&P 500 index in their own portfolios.

> From [NewPortQuant](): "IVV is one of the three largest S&P 500 ETFs (the other two are SPY and VOO). IVV provides daily holding reports since March 24, 2017, and end-of-month holding reports since September 2006."

If you click on the link in the cell below, you will be prompted to download a csv file containing the data of the S&P 500 components in Oct 31, 2006. To download the data for the entire time period of this study, I just need to loop over the list I created previously, and replace the last part of this link with the dates on my list (without any separators).

https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf/1467271812596.ajax?fileType=csv&fileName=IVV_holdings&dataType=fund&asOfDate=20061031

In many time periods, the S&P 500 contains more than 500 tickers of companies (mostly because of different share classes of stocks for the same company). The number of tickers of the S&P 500 varies between 500 and 519, so I created the `complete_with_nan` function to make every pandas series (with the S&P 500 tickers) have exactly 520 lines, so there would be no issues later when I try to concatenate all these series in a pandas dataframe.

Inside the loop, I use the `skiprows = 9` option in the `pd.read_csv` funcion, and then select `[:-2]` and `['Ticker']` to clean the data and get only the tickers column.

In [7]:
# Creating the monthly dataframe with S&P 500 tickers
# We will download the csv files with the tickers from ishares website

# def complete_with_nan(pandas_series):
#     """Complete a pandas series with NaN until it has 520 lines"""
#     i = len(pandas_series)
#     while i < 520:
#         pandas_series[i] = np.nan
#         i += 1

# url = r"https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf/1467271812596.ajax?fileType=csv&fileName=IVV_holdings&dataType=fund&asOfDate="

# df_sp500_tickers = pd.DataFrame()

# # Dowload, clean and store the tickers of S&P 500 each month
# for end_of_month in end_of_month_dates:
#     tickers = pd.read_csv(url + str(end_of_month).replace('-', ''),
#                           skiprows = 9)[:-2]['Ticker']
#     tickers.name = end_of_month
#     complete_with_nan(tickers)
#     df_sp500_tickers = pd.concat([df_sp500_tickers, tickers], axis = 1)

# df_sp500_tickers

In [8]:
# Uncomment to store the data locally in the current directory (~ 661 KB)
# df_sp500_tickers.to_pickle('df_sp500_tickers.pkl')

In [9]:
# Uncomment to read the data stored locally
df_sp500_tickers = pd.read_pickle('df_sp500_tickers.pkl')
df_sp500_tickers

,2006-10-31,2006-11-30,2006-12-29,2007-01-31,2007-02-28,2007-03-30,2007-04-30,2007-05-31,2007-06-29,2007-07-31,...,2022-03-31,2022-04-29,2022-05-31,2022-06-30,2022-07-29,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-30
0,PMCS,PMCS,PMCS,PMCS,PMCS,BGIXX,PMCS,PMCS,PMCS,CCTYQ,...,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,NaN,NaN,NaN
1,PGL,ADCT,ADCT,PGL,ADCT,PMCS,SANM,SANM,SANM,GAS.2,...,MSFT,MSFT,MSFT,MSFT,MSFT,MSFT,MSFT,NaN,NaN,NaN
2,ADCT,PGL,PGL,SANM,SANM,SANM,DDS,ADCT,GAS.2,DDS,...,AMZN,AMZN,AMZN,AMZN,AMZN,AMZN,AMZN,NaN,NaN,NaN
3,NAV,NAV,SANM,ADCT,DDS,ADCT,ADCT,GAS.2,CCTYQ,QLG,...,TSLA,TSLA,GOOGL,GOOGL,TSLA,TSLA,TSLA,NaN,NaN,NaN
4,DDS,SANM,PMTC,GAS.2,GAS.2,DDS,HPC,HPC,ADCT,MDP,...,GOOGL,GOOGL,TSLA,GOOG,GOOGL,GOOGL,GOOGL,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Done. I have the tickers of the S&P 500 for most of my study time period. But there are still missing data for some months:

In [10]:
a = df_sp500_tickers.isnull().sum()
display(a[a == 520])

2010-05-31    520
2017-01-31    520
2017-02-28    520
2017-03-31    520
2017-04-28    520
2017-05-31    520
2017-06-30    520
2022-10-31    520
2022-11-30    520
2022-12-30    520
dtype: int64

The series above show the months that have only NaN. The final months of 2022 in the list have NaNs because they are in the future at the time I am writing this, so obviously there is no data yet.

Other than that, there is missing data in ishares website for some months. To deal with those, I will generate the composition of the S&P 500 for those months based on the public data of the changes in the S&P 500 components, available at the [NewPortQuant website](https://newportquant.com/historical-components-of-sp-500-index/). 

To download the csv file, click on the link above the table on that web page (or click [here](https://docs.google.com/spreadsheets/d/1xkq2kkf-iElKl9BhEwqQx3Pgkh0B9dFKJpefQ4oOI_g/edit#gid=455032226)), then click File -> Download -> Comma Separated Values (.csv). Then put the csv file in the same directory of this notebook, in case you are running it locally.

In [11]:
changes_sp500 = pd.read_csv('sp500change - sp500change.csv',
                            skiprows = 2, parse_dates = [0, 1])
changes_sp500

,DateBeforeChange,DateAfterChange,Change,Ticker,Name,Sector,Exchange
0,2022-02-28,2022-03-01,1,MOH,MOLINA HEALTHCARE INC,Health Care,New York Stock Exchange Inc.
1,2022-02-25,2022-02-28,-1,INFO,IHS MARKIT LTD,Industrials,New York Stock Exchange Inc.
2,2022-02-16,2022-02-17,1,PARA,PARAMOUNT GLOBAL CLASS B,Communication,NASDAQ
3,2022-02-16,2022-02-17,-1,VIAC,VIACOMCBS INC CLASS B,Communication,NASDAQ
4,2022-02-11,2022-02-14,1,NDSN,NORDSON CORP,Industrials,NASDAQ
...,...,...,...,...,...,...,...
973,2006-10-31,2006-11-30,-1,LPX,LOUISIANA PACIFIC CORP,-,New York Stock Exchange Inc.
974,2006-09-29,2006-10-31,1,SII,SMITH INTERNATIONAL,-,New York Stock Exchange Inc.
975,2006-09-29,2006-10-31,1,WU,WESTERN UNION,-,New York Stock Exchange Inc.
976,2006-09-29,2006-10-31,-1,ANDW,ANDREW CORP.,-,NASDAQ


This table shows the ticker, when the change occurred, and whether the company was removed (Change = -1), or added (Change = 1) to the S&P 500 index. With this data, I can generate the data that was missing in the `df_sp500_tickers` dataframe.

In [12]:
changes_sp500.loc[changes_sp500.DateAfterChange >= datetime(2010, 4, 30)].loc[changes_sp500.DateAfterChange <= datetime(2010, 6, 30)]

,DateBeforeChange,DateAfterChange,Change,Ticker,Name,Sector,Exchange
727,2010-05-28,2010-06-30,1,KMX,CARMAX INC,Consumer Discretionary,New York Stock Exchange Inc.
728,2010-05-28,2010-06-30,-1,XTO,XTO ENERGY INC,Energy,New York Stock Exchange Inc.
729,2010-03-31,2010-04-30,1,CERN,CERNER CORP,Health Care,NASDAQ
730,2010-03-31,2010-04-30,-1,BJS,BJ SERVICES CO,Energy,New York Stock Exchange Inc.


Those are the changes from April 2010 to June 2010. So to generate the S&P 500 composition for '2010-05-31', I just need to copy from '2010-04-30', since there were changes only in April and June, but not in May (`DateAfterChange` column).

In [13]:
df_sp500_tickers[date(2010, 5, 31)] = df_sp500_tickers[date(2010, 4, 30)]
a = df_sp500_tickers.isnull().sum()
display(a[a == 520])

2017-01-31    520
2017-02-28    520
2017-03-31    520
2017-04-28    520
2017-05-31    520
2017-06-30    520
2022-10-31    520
2022-11-30    520
2022-12-30    520
dtype: int64

Now I just need to generate the missing data between January and June of 2017. Below are the changes in the S&P 500 composition during this period.

In [14]:
changes_sp500.loc[changes_sp500.DateAfterChange >= datetime(2017, 1, 1)].loc[changes_sp500.DateAfterChange <= datetime(2017, 6, 30)]

,DateBeforeChange,DateAfterChange,Change,Ticker,Name,Sector,Exchange
268,2017-06-15,2017-06-16,1,ALGN,ALIGN TECHNOLOGY INC,Health Care,NASDAQ
269,2017-06-15,2017-06-16,1,ANSS,ANSYS INC,Information Technology,NASDAQ
270,2017-06-15,2017-06-16,1,HLT,HILTON WORLDWIDE HOLDINGS INC,Consumer Discretionary,New York Stock Exchange Inc.
271,2017-06-15,2017-06-16,1,RE,EVEREST RE GROUP LTD,Financials,New York Stock Exchange Inc.
272,2017-06-15,2017-06-16,-1,R,RYDER SYSTEM INC,Industrials,New York Stock Exchange Inc.
273,2017-06-15,2017-06-16,-1,TDC,TERADATA CORP,Information Technology,New York Stock Exchange Inc.
274,2017-06-15,2017-06-16,-1,YHOO,YAHOO! INC,Information Technology,NASDAQ
275,2017-06-14,2017-06-15,-1,MJN,MEAD JOHNSON NUTRITION,Consumer Staples,New York Stock Exchange Inc.
276,2017-05-31,2017-06-01,1,INFO,IHS MARKIT LTD,Industrials,NASDAQ
277,2017-05-31,2017-06-01,-1,TGNA,TEGNA INC,Consumer Discretionary,New York Stock Exchange Inc.


When a company/ticker is removed from the S&P 500, another one is added at the same time. Then, to generate the composition for the months with missing data, I just need to copy the data from the previous month, then find the index of the tickers that were removed, then replace their values (strings) with those that were added (in that respective month), according to the dataframe above. 

The order of the tickers in the same month/column in `df_sp500_tickers` does not matter for this project.

In [15]:
df_sp500_tickers[date(2017, 1, 31)] = df_sp500_tickers[date(2016, 12, 30)]
a = df_sp500_tickers[date(2017, 1, 31)] == 'STJ'
a[a == True]

207    True
Name: 2017-01-31, dtype: bool

In [16]:
df_sp500_tickers[date(2017, 1, 31)][207]

'STJ'

In [17]:
df_sp500_tickers[date(2017, 1, 31)][207] = 'IDXX'
df_sp500_tickers[date(2017, 1, 31)][207]

'IDXX'

The S&P 500 composition in 2017-01-31 is now succesfully generated. I need to do the same thing for the remaining missing data of 2017 (Feb to June). But since this is a repetitive task of finding and replacing/updating values in a pandas series, I created the `return_index_list`, and the `replace_sp500_components` functions to keep things organized.

In [18]:
def return_index_list(tickers_to_search, pandas_series):
    """
    Function to search for the values of a list on a pandas series,
    and return a list with the indexes of the matching values.
    
    tickers_to_search: list with the values to search for.
    pandas_series: dataset where the search will be conducted.
    """
    index_list = []
    
    for ticker_idx in pandas_series.index:
        if pandas_series[ticker_idx] in tickers_to_search:
            index_list.append(ticker_idx)
    
    return index_list

In [19]:
# Function to replace the S&P 500 components
def replace_sp500_components(list_of_indexes, list_of_new_tickers, pandas_series):
    """
    This function replaces the values of a series with the new ones provided.
    It also prints the before and after situation.
    
    list_of_indexes: indexes of the elements to be replaced in pandas_series.
    list_of_new_tickers: list of str, has to be the same len as list_of_indexes.
    pandas_series: dataset where the changes will occur.
    """
    
    print('Before change:')
    for index in list_of_indexes:
        print(index, pandas_series[index])
    print()
    
    for n, index in enumerate(list_of_indexes):
        pandas_series[index] = list_of_new_tickers[n]
    
    print('After change:')
    for index in list_of_indexes:
        print(index, pandas_series[index])

In [20]:
df_sp500_tickers[date(2017, 2, 28)] = df_sp500_tickers[date(2017, 1, 31)]

old_tickers = ['SE', 'PBI']

index_list = return_index_list(old_tickers,
                               df_sp500_tickers[date(2017, 2, 28)])

new_tickers = ['CBOE', 'INCY']

replace_sp500_components(index_list, new_tickers, 
                         df_sp500_tickers[date(2017, 2, 28)])

Before change:
162 SE
503 PBI

After change:
162 CBOE
503 INCY


In [21]:
a = df_sp500_tickers.isnull().sum()
display(a[a == 520])

2017-03-31    520
2017-04-28    520
2017-05-31    520
2017-06-30    520
2022-10-31    520
2022-11-30    520
2022-12-30    520
dtype: int64

In [22]:
df_sp500_tickers[date(2017, 3, 31)] = df_sp500_tickers[date(2017, 2, 28)]

old_tickers = ['URBN', 'LLTC', 'HAR', 'FTR', 'FSLR', 'ENDP']

index_list = return_index_list(old_tickers,
                               df_sp500_tickers[date(2017, 3, 31)])

new_tickers = ['SNPS', 'RJF', 'REG', 'DISH', 'ARE', 'AMD']

replace_sp500_components(index_list, new_tickers,
                         df_sp500_tickers[date(2017, 3, 31)])

Before change:
286 LLTC
442 HAR
497 FTR
501 ENDP
504 FSLR
505 URBN

After change:
286 SNPS
442 RJF
497 REG
501 DISH
504 ARE
505 AMD


In [23]:
a = df_sp500_tickers.isnull().sum()
display(a[a == 520])

2017-04-28    520
2017-05-31    520
2017-06-30    520
2022-10-31    520
2022-11-30    520
2022-12-30    520
dtype: int64

In [24]:
changes_sp500.loc[changes_sp500.DateAfterChange >= datetime(2017, 4, 1)].loc[changes_sp500.DateAfterChange <= datetime(2017, 6, 30)]

,DateBeforeChange,DateAfterChange,Change,Ticker,Name,Sector,Exchange
268,2017-06-15,2017-06-16,1,ALGN,ALIGN TECHNOLOGY INC,Health Care,NASDAQ
269,2017-06-15,2017-06-16,1,ANSS,ANSYS INC,Information Technology,NASDAQ
270,2017-06-15,2017-06-16,1,HLT,HILTON WORLDWIDE HOLDINGS INC,Consumer Discretionary,New York Stock Exchange Inc.
271,2017-06-15,2017-06-16,1,RE,EVEREST RE GROUP LTD,Financials,New York Stock Exchange Inc.
272,2017-06-15,2017-06-16,-1,R,RYDER SYSTEM INC,Industrials,New York Stock Exchange Inc.
273,2017-06-15,2017-06-16,-1,TDC,TERADATA CORP,Information Technology,New York Stock Exchange Inc.
274,2017-06-15,2017-06-16,-1,YHOO,YAHOO! INC,Information Technology,NASDAQ
275,2017-06-14,2017-06-15,-1,MJN,MEAD JOHNSON NUTRITION,Consumer Staples,New York Stock Exchange Inc.
276,2017-05-31,2017-06-01,1,INFO,IHS MARKIT LTD,Industrials,NASDAQ
277,2017-05-31,2017-06-01,-1,TGNA,TEGNA INC,Consumer Discretionary,New York Stock Exchange Inc.


In [25]:
df_sp500_tickers[date(2017, 4, 28)] = df_sp500_tickers[date(2017, 3, 31)]

old_tickers = ['SWN', 'DNB', 'BF.B']

index_list = return_index_list(old_tickers,
                               df_sp500_tickers[date(2017, 4, 28)])

new_tickers = ['IT', 'DXC', 'BFB']

replace_sp500_components(index_list, new_tickers,
                         df_sp500_tickers[date(2017, 4, 28)])

Before change:
430 BF.B
483 SWN
491 DNB

After change:
430 IT
483 DXC
491 BFB


In [26]:
df_sp500_tickers[date(2017, 5, 31)] = df_sp500_tickers[date(2017, 4, 28)]

In [27]:
a = df_sp500_tickers.isnull().sum()
display(a[a == 520])

2017-06-30    520
2022-10-31    520
2022-11-30    520
2022-12-30    520
dtype: int64

In [28]:
df_sp500_tickers[date(2017, 6, 30)] = df_sp500_tickers[date(2017, 5, 31)]

old_tickers = ['TGNA', 'MJN', 'YHOO', 'TDC', 'R']

index_list = return_index_list(old_tickers,
                               df_sp500_tickers[date(2017, 6, 30)])

new_tickers = ['INFO', 'RE', 'HLT', 'ANSS', 'ALGN']

replace_sp500_components(index_list, new_tickers,
                         df_sp500_tickers[date(2017, 6, 30)])

Before change:
135 YHOO
324 MJN
492 TGNA
496 R
499 TDC

After change:
135 INFO
324 RE
492 HLT
496 ANSS
499 ALGN


In [29]:
a = df_sp500_tickers.isnull().sum()
display(a[a == 520])

2022-10-31    520
2022-11-30    520
2022-12-30    520
dtype: int64

In [30]:
df_sp500_tickers.head(10)

,2006-10-31,2006-11-30,2006-12-29,2007-01-31,2007-02-28,2007-03-30,2007-04-30,2007-05-31,2007-06-29,2007-07-31,...,2022-03-31,2022-04-29,2022-05-31,2022-06-30,2022-07-29,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-30
0,PMCS,PMCS,PMCS,PMCS,PMCS,BGIXX,PMCS,PMCS,PMCS,CCTYQ,...,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,NaN,NaN,NaN
1,PGL,ADCT,ADCT,PGL,ADCT,PMCS,SANM,SANM,SANM,GAS.2,...,MSFT,MSFT,MSFT,MSFT,MSFT,MSFT,MSFT,NaN,NaN,NaN
2,ADCT,PGL,PGL,SANM,SANM,SANM,DDS,ADCT,GAS.2,DDS,...,AMZN,AMZN,AMZN,AMZN,AMZN,AMZN,AMZN,NaN,NaN,NaN
3,NAV,NAV,SANM,ADCT,DDS,ADCT,ADCT,GAS.2,CCTYQ,QLG,...,TSLA,TSLA,GOOGL,GOOGL,TSLA,TSLA,TSLA,NaN,NaN,NaN
4,DDS,SANM,PMTC,GAS.2,GAS.2,DDS,HPC,HPC,ADCT,MDP,...,GOOGL,GOOGL,TSLA,GOOG,GOOGL,GOOGL,GOOGL,NaN,NaN,NaN
5,CIEN,MDP,NOVL,DDS,NOVL,GAS.2,MDP,DDS,DDS,NOVL,...,GOOG,GOOG,GOOG,TSLA,GOOG,GOOG,GOOG,NaN,NaN,NaN
6,GAS.2,CIEN,GAS.2,HPC,MDP,MDP,GAS.2,CCTYQ,HPC,TRB,...,NVDA,BRKB,BRKB,BRKB,BRKB,BRKB,BRKB,NaN,NaN,NaN
7,NOVL,NOVL,DDS,MDP,HPC,DJ,DJ,MDP,MDP,HPC,...,BRKB,UNH,JNJ,UNH,UNH,UNH,UNH,NaN,NaN,NaN
8,MDP,HPC,MDP,CIEN,DJ,HPC,TEK,TEK,TRB,KBH,...,FB,JNJ,UNH,JNJ,JNJ,JNJ,JNJ,NaN,NaN,NaN
9,HPC,DDS,HPC,TEK,TEK,TEK,CIEN,NOVL,QLG,THC,...,UNH,NVDA,NVDA,NVDA,NVDA,XOM,XOM,NaN,NaN,NaN


#### The dataframe with S&P 500 tickers is now complete!

## 4. Getting data from yahoo finance

In [31]:
# Creating a list with all the tickers from df_sp500_tickers

sp500_list = pd.Series(dtype = 'object')

for index in range(len(df_sp500_tickers.columns)):
    sp500_list = pd.concat([sp500_list, df_sp500_tickers.iloc[:, index]])

sp500_list.dropna(inplace = True)
sp500_list.drop_duplicates(inplace = True)
sp500_list = sp500_list.values
sp500_list

array(['PMCS', 'PGL', 'ADCT', ..., 'CSGP', 'INVH', 'ESZ2'], dtype=object)

In [32]:
# Getting data from yahoo finance

# df_prices = pdr.get_data_yahoo(symbols = sp500_list,
#                                start = '2006-10-01',
#                                end = '2022-12-31')['Adj Close']

In [33]:
# Uncomment to save the data to disk (~ 32.5 MB)
# df_prices.to_pickle('df_prices.pkl')

In [34]:
# Uncomment to read the data from the disk
df_prices = pd.read_pickle('df_prices.pkl')

In [35]:
df_prices

Symbols,PGL,ADCT,DDS,CIEN,MDP,HPC,SANM,LPX,UIS,DYN,...,ESH1,MPWR,GNRC,ESM1,ESU1,ESZ1,ESH2,ESM2,ESU2,ESZ2
Date,,,,,,,,,,,,,,,,,,,,,
2006-10-31,NaN,NaN,24.362997,23.510000,NaN,NaN,23.700001,17.041698,65.400002,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-11-01,NaN,NaN,24.048059,22.170000,NaN,NaN,23.280001,16.826311,66.099998,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-11-02,NaN,NaN,23.652386,22.129999,NaN,NaN,23.219999,16.955538,66.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-11-03,NaN,NaN,23.208237,22.610001,NaN,NaN,23.400000,16.481684,67.500000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-11-06,NaN,NaN,23.579710,23.340000,NaN,NaN,23.879999,16.516148,68.099998,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-20,NaN,4.68,298.320007,40.889999,NaN,NaN,52.750000,52.849998,8.590000,11.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-21,NaN,4.78,310.880005,43.080002,NaN,NaN,55.320000,55.740002,8.810000,11.78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-24,NaN,4.55,319.910004,43.689999,NaN,NaN,56.200001,56.169998,8.790000,11.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_prices.isnull().sum()[df_prices.isnull().sum() > 0]

Symbols
PGL     1891
ADCT    3408
MDP     4024
HPC     3185
DYN     3493
        ... 
ESZ1    4025
ESH2    4025
ESM2    4025
ESU2    4025
ESZ2    4025
Length: 587, dtype: int64

In [37]:
df_prices.isnull().sum()[df_prices.isnull().sum() > 1000]

Symbols
PGL     1891
ADCT    3408
MDP     4024
HPC     3185
DYN     3493
        ... 
ESZ1    4025
ESH2    4025
ESM2    4025
ESU2    4025
ESZ2    4025
Length: 560, dtype: int64

In [38]:
df_prices.isnull().sum()[df_prices.isnull().sum() > 2000]

Symbols
ADCT    3408
MDP     4024
HPC     3185
DYN     3493
CPWR    2677
        ... 
ESZ1    4025
ESH2    4025
ESM2    4025
ESU2    4025
ESZ2    4025
Length: 497, dtype: int64

In [39]:
df_prices.isnull().sum()[df_prices.isnull().sum() > 3000]

Symbols
ADCT    3408
MDP     4024
HPC     3185
DYN     3493
HMA     3867
        ... 
ESZ1    4025
ESH2    4025
ESM2    4025
ESU2    4025
ESZ2    4025
Length: 461, dtype: int64

In [40]:
df_prices.isnull().sum()[df_prices.isnull().sum() >= 4024]

Symbols
MDP     4024
CVH     4024
LKQ     4024
DISH    4024
PMCS    4025
        ... 
ESZ1    4025
ESH2    4025
ESM2    4025
ESU2    4025
ESZ2    4025
Length: 418, dtype: int64

It seems unfortunately there is a lot of missing data